In [1]:
%matplotlib widget
from ursjupyter import *
from smvjupyter import *

usfs_sites = "docs/usfs_sites/Sites_lf_geo.json"                      # USFS sites
usfs_regions = "docs/usfs_admin/USFS_Regional_Boundaries.json"        # USFS administrative regions

try:
    app = JupyterSMV(usfs_sites, session=session)
except:
    print("You must be logged with Earthdata to use this widget."
          "Login with the cell above, then run this cell again.")

'Login successful. Download with: session.get(url)'

In [ ]:
layers = app.layers
layers

In [ ]:
layer = layers.iloc[1]
layer

In [ ]:
xrdataset = layer.xr.sel(stat="Mean", drop=True)
xrdataset

In [ ]:
def fDaymet(xrds, Interval=None, Stack=False):
    """ """
    
    series = [[xrds.tmin, 0, dict(color="darkorange", label="avg min temp")],
              [xrds.tmax, 0, dict(color="darkgreen", label="avg max temp")],
              [xrds.prcp, 1, dict(color="purple", label="avg rate of precip")]]
    
    if (Stack) & (Interval!="year"):
        for i, l in enumerate(series):
            series[i][0] = series[i][0].mean("year", skipna=True)
            series[i][2]["x"] = Interval
    
    fmt0 = [lambda a: a.grid("on", alpha=0.5),
            lambda a: a.legend(loc="upper left"),
            lambda a: a.set_ylabel("degrees Celsius")]
    fmt1 = [lambda a: a.legend(loc="upper right"), 
            lambda a: a.set_ylabel("mm/day")] 

    return((series, fmt0, fmt1))


def fProductivity(xrds, Interval=None, Stack=False):
    """ """
    
    series = [[xrds.GPP_mean, 0, dict(color="darkorange", label="GPP")],
              [xrds.NEE_mean, 0, dict(color="darkgreen", label="NEE")]]
    
    if (Stack) & (Interval!="year"):
        for i, l in enumerate(series):
            series[i][0] = series[i][0].mean("year", skipna=True)
            series[i][2]["x"] = Interval
    
    fmt0 = [lambda a: a.grid("on", alpha=0.5),
            lambda a: a.legend(loc="upper left"),
            lambda a: a.set_ylabel("g m-2 d-1")]
    fmt1 = [lambda a: a.set_title(None)]

    return((series, fmt0, fmt1))


def fSoilMoisture(xrds, Interval=None, Stack=False):
    """ """
    
    fyear = lambda x: x.sel(year=np.unique(x.year)).mean("year")
    
    series = []
    for d in ["surface", "rootzone"]:
        ds = xrds.filter_by_attrs(soil_zone=d)
        ds = xr.concat(ds.values(), "mean").mean("mean", skipna=True)
        
        p = [ds, 0, dict(label=d+" mean")]
        if (Stack)&(Interval!="year"):
            p[0] = fyear(ds)
            p[2]["x"] = Interval
        else:
            p[2]["x"] = "year" if (Stack)&(Interval=="year") else "time"
            
        series.append(p) 
    
    fmt0 = [lambda a: a.set_title(None),
            lambda a: a.grid("on", alpha=0.5),
            lambda a: a.legend(loc="upper left"),
            lambda a: a.set_ylabel("soil moisture volume (m3/m3)")]
    fmt1 = [lambda a: a.set_title(None)]

    return((series, fmt0, fmt1))

In [ ]:
class Plotter:
    """Generates the map/plot side-by-side widget container."""
    
    error = HTML("""<p>Selections cannot be plotted <b>""" 
                 """(probably time slider)</b>.</p>""")
    wait = HTML("""<p style="position:relative;top:50%;transform:translateY"""
                """(-50%);">Calculating statistics. Please be patient.</p>""")
    
    dataset_keys = {
        "Soil Moisture": (dict(units="m3/m3"), fSoilMoisture),
        "SMAP GPP/NEE": (dict(units="g m-2 d-1"), fProductivity),
        "Daymet": (dict(source="Daymet"), fDaymet)}
    
    interval_keys = {
        "day": ("d", "dayofyear"), 
        "week": ("w", "week"), 
        "month": ("m", "month"), 
        "year": ("y", "year")}

    
    def __init__(self, layer): 

        self.layer = layer        
        self.xr = layer.xr.sel(stat="Mean", drop=True)
        self.time = pd.to_datetime(self.xr.time.data).strftime("%Y-%m-%d")
        self.samp = layer.samples.samp.tolist()

        # --------------------------------------------------------------------
        # init widgets and set interactivity

        self.dsel = Dropdown(
            options=["Soil Moisture", "SMAP GPP/NEE", "Daymet"],
            value="Soil Moisture",
            layout=Layout(width='auto'))
        self.dsel.observe(self.handler, names="value")
        
        self.intv = Dropdown(
            options=self.interval_keys.keys(), 
            value="day",
            layout=Layout(width='auto'))
        self.intv.observe(self.handler, names="value")
        
        self.stack = ToggleButton(description="Stack", value=False)
        self.stack.observe(self.update_stack, names="value")
        
        self.dtrange = SelectionRangeSlider(
            options=self.time,
            value=[self.time[0], self.time[-1]],
            continuous_update=False,
            layout=Layout(width="50%"))
        self.dtrange.observe(self.update_time, names="value")
        
        # --------------------------------------------------------------------
        # construct ui
        
        self.fig, self.ax0 = plt.subplots(figsize=(7, 3.5))
        self.ax1 = self.ax0.twinx()
        self.fig.tight_layout(pad=2, h_pad=4)
            
        self.mapw = Map(
            layers=(self.layer.layer.layer, layer.points, bmap,), 
            center=(layer.lat, layer.lon), 
            zoom=9, attribution_control=False)
        for p in layer.points.layers:
            p.on_click(self.update_sample)

        self.selections = Box(
            children=[self.dsel, self.intv, self.stack, self.dtrange], 
            layout=Layout(
                width="100%",
                display='flex',
                flex_flow='row',
                align_items='stretch'))
        
        self.outputs = GridBox(
            children=[self.mapw, self.fig.canvas], 
            layout=Layout(
                width='100%',
                grid_template_rows="auto",
                grid_template_columns="25% 75%",
                grid_template_areas='''"self.mapw self.fig.canvas"'''))
        
        self.ui = VBox([self.selections, self.outputs])
        display(self.ui)
        
        # --------------------------------------------------------------------        
        # trigger draw
        
        self.handler()

    # ------------------------------------------------------------------------
    
    def update_sample(self, **kwargs):
        """ """
        self.handler()
    
    def update_dataset(self, change):
        """ """
        if (not self.stack.value)&(self.intv.value=="day"):
            self.dtrange.value = [self.time[0], self.time[-1]]
        self.handler()
    
    def update_stack(self, change):
        """ """
        stack = change.new
        if stack: 
            self.dtrange.disabled = True
        else: 
            self.dtrange.disabled = False
        self.handler()

    def update_time(self, change):
        """ """
        start, end = change.new
        self.ax0.set_xlim(start, end)
        self.fig.canvas.draw()
        
    def handler(self, change=None):
        """ """
        dataset = self.dsel.value
        interval = self.intv.value
        stack = self.stack.value
        
        # select toggled-on samples
        samp = [s.id for s in self.samp if s.on]
        ds = self.xr.sel(sample=samp)
        
        # filter to dataset-level
        dfilter, dfunc = self.dataset_keys[self.dsel.value]
        ds = ds.filter_by_attrs(**dfilter)
        ds = ds.mean("sample", skipna=True, keep_attrs=True)
        
        # aggregate to interval and 
        intvlstr, stackstr = self.interval_keys[interval]
        if interval!="day":
            ds = ds.resample(time="1"+intvlstr).mean(skipna=True, keep_attrs=True) 
            
        ds.coords["year"] = ds.time.dt.year
        sfunc = lambda x: x.groupby(interval).mean(skipna=True, keep_attrs=True)
        if stack:
            ds.coords[interval] = getattr(ds.time.dt, stackstr)
            ds = ds.groupby("year").apply(sfunc)

        self.plot_config = dfunc(ds, Interval=interval, Stack=stack)
        self.plotter()
        #try:
        #    self.plotter()
        #except:
        #    print("Something went wrong. Try again.")


    def plotter(self):
        """ """
        self.ax0.clear(); self.ax1.clear()
        
        series, fmt0, fmt1 = self.plot_config
        #count0,count1 = 0,0
        for i, s in enumerate(series):
            x, xax, xargs = s
            ax = self.ax0 if xax==0 else self.ax1
            #if xax==0: 
            #    count0 += 1
            #else: 
            #    count1 += 1
            x.plot(ax=ax, **xargs)
        
        for c in fmt0: c(self.ax0)
        for c in fmt1: c(self.ax1)
        #if count1==0: self.ax1.axis("off")
        #else: self.ax1.axis("on")
        
        self.fig.canvas.draw()


In [ ]:
p = Plotter(layer)

In [ ]:
p.xr["time"]

### backup

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

"""
def plotDaymet(xrds, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(10, 4.5))
    fig.tight_layout()
    ax1 = ax0.twinx()
   
    prcp = xrds.prcp
    tmin = xrds.tmin
    tmax = xrds.tmax
    ylabels = ("degrees Celsius", "mm/day")
    
    if (Stack) & (Interval!="year"):
        l1 = prcp.mean("year").plot.line(x=Interval, ax=ax0, color="darkorange", label="avg rate of precip")
        l2 = tmin.mean("year").plot.line(x=Interval, ax=ax0, color="darkgreen", label="avg min temp")
        l3 = tmax.mean("year").plot.line(x=Interval, ax=ax1, color="purple", label="avg max temp", linestyle="--")
    else:
        l1 = prcp.plot(ax=ax0, color="darkorange", label="avg rate of precip")
        l2 = tmin.plot(ax=ax0, color="darkgreen", label="avg min temp")
        l3 = tmax.plot(ax=ax1, color="purple", label="avg max temp", linestyle="--")

    ax0.grid("on", alpha=0.5)
    ax0.legend(loc="upper left"); ax1.legend()
    ax0.set_ylabel(ylabels[0]); ax1.set_ylabel(ylabels[1])

    plt.show()
    

def plotSoilMoisture(xrds, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(10, 4.5)) 
    fig.tight_layout()
    ax1 = ax0.twinx()

    for plotter in [
        ("surface", xrds.filter_by_attrs(soil_zone="surface")), 
        ("rootzone", xrds.filter_by_attrs(soil_zone="rootzone"))]:
        
        label = plotter[0]+" mean"
        
        if (Stack) & (Interval!="year"):
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").sel(year=np.unique(stack.year))
            pmean.mean("year").plot.line(x=Interval, ax=ax0, label=label)
        elif (Stack) & (Interval=="year"):
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").plot(x="year", ax=ax0, label=label) 
        else:
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").plot(x="time", ax=ax0, label=label)

        #ax.fill_between(pmean.time.data, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    ax0.set_ylabel("soil moisture volume (m3/m3)")
    ax0.grid("on", alpha=0.25)
    ax0.legend(loc=2)
    ax0.set_title(None)
    ax0.set_xlabel(Interval)
        
    plt.show()
"""


def fDaymet(xrds, var=None, Interval=None, Stack=False):
    """ """
    
    series = [[xrds.tmin, 0, dict(color="darkorange", label="avg min temp")],
              [xrds.tmax, 0, dict(color="darkgreen", label="avg max temp")],
              [xrds.prcp, 1, dict(color="purple", label="avg rate of precip", linestyle="--")]]
    
    if (Stack) & (Interval!="year"):
        for i, l in enumerate(series):
            series[i][0] = series[i][0].mean("year")
            series[i][2]["x"] = Interval
    
    fmt0 = [lambda a: a.grid("on", alpha=0.5),
            lambda a: a.legend(loc="upper left"),
            lambda a: a.set_ylabel("degrees Celsius")]
    fmt1 = [lambda a: a.legend(loc="upper right"), 
            lambda a: a.set_ylabel("mm/day")] 

    return((series, fmt0, fmt1))
    

def fSoilMoisture(xrds, Interval=None, Stack=False):
    """ """
    
    surface = xr.concat(xrds.filter_by_attrs(soil_zone="surface"), "mean").mean("mean")
    rootzone = xr.concat(xrds.filter_by_attrs(soil_zone="rootzone"), "mean").mean("mean")
    
    fyear = lambda x: x.sel(year=np.unique(x.year)).mean("year")
    
    series = []
    for d in [(surface, "surface"), (rootzone, "rootzone")]:
        p = [d[0], 0, dict(label=d[1]+" mean")]
        if (Stack)&(Interval!="year"):
            p[0] = fyear(p[0])
            p[2]["x"] = Interval
        else:
            p[2]["x"] = "year" if (Stack)&(Interval=="year") else "time"
        series.append(p) 
    
    fmt0 = [lambda a: a.set_title(None),
            lambda a: a.grid("on", alpha=0.5),
            lambda a: a.legend(loc="upper left"),
            lambda a: a.set_ylabel("soil moisture volume (m3/m3)")]
    fmt1 = [] 

    return((series, fmt0, fmt1))


class Plotter:
    """Generates the map/plot side-by-side widget container."""
    
    error = HTML("""<p>Selections cannot be plotted <b>(probably time slider)</b>.</p>""")
    wait = HTML("""<p style="position: relative; top: 50%; transform: translateY"""
                """(-50%);">Calculating statistics. Please be patient.</p>""")
    
    dataset_keys = {
        "Daymet": (lambda x: x.filter_by_attrs(source="Daymet"), plotDaymet), 
        "Soil moisture": (lambda x: x.filter_by_attrs(units="m3/m3"), plotSoilMoisture)}
    
    interval_keys = {
        "day": ("d", "dayofyear"), 
        "week": ("w", "week"), 
        "month": ("m", "month"), 
        "year": ("y", "year")}

    
    def __init__(self, layer): 

        self.layer = layer        
        self.xr = layer.xr.sel(stat="Mean", drop=True)
        self.time = pd.to_datetime(self.xr.time.data).strftime("%Y-%m-%d")

        # --------------------------------------------------------------------
        # init widgets and set interactivity

        self.dsel = Dropdown(
            options=["Soil moisture", "GPP/NEE", "Daymet"], 
            value="Soil moisture",
            layout=Layout(width='auto'))
        
        self.intv = Dropdown(
            options=self.interval_keys.keys(), 
            value="day",
            layout=Layout(width='auto'))
        
        self.stack = ToggleButton(description="Stack", value=False)
        
        self.status = Output()
        
        self.dtrange = SelectionRangeSlider(
            options=self.time,
            value=[self.time[0], self.time[-1]],
            continuous_update=False,
            layout=Layout(width="auto"))
        
        self.w = interactive(
            self.handler, 
            Dataset=self.dsel, 
            Interval=self.intv, 
            Stack=self.stack,
            Time=self.dtrange,
            layout=Layout(
                width="100px",
                display='flex',
                flex_flow='column',
                align_items='stretch'))
        
        # --------------------------------------------------------------------
        # construct ui
        
        self.mapw = Map(
            layers=(self.layer.layer.layer, layer.points, bmap,), 
            center=(layer.lat, layer.lon), 
            zoom=9, attribution_control=False)   

        self.topselect = Box(
            children=[self.dsel, self.intv, self.stack], 
            layout=Layout(
                width="auto",
                display='flex',
                flex_flow='row',
                align_items='stretch'))
        
        self.output = Output(layout={"width": "auto", "height": "340px"})
        self.ui = HBox([self.mapw, VBox([
            self.topselect, 
            self.output, 
            self.dtrange
        ], layout=Layout(width="75%"))])
        display(self.ui)
        
        # --------------------------------------------------------------------        
        # init plot canvas and trigger draw
        
        self.fig, self.ax0 = plt.subplots(figsize=(10, 4.5)) 
        self.ax1 = self.ax0.twinx()
        self.fig.tight_layout()
        
        # draw init
        self.handler(
            self.dsel.value, 
            self.intv.value, 
            self.stack.value,
            self.dtrange.value)

    # ------------------------------------------------------------------------
    
    def handler(self, Dataset, Interval, Stack, Time):
        """ """
        
        self.output.clear_output()
        with self.output:
            display(self.wait)
        
        dfilt, pfunc = self.dataset_keys[Dataset]
        ds = dfilt(self.xr)
        ds = ds.sel(time=slice(Time[0], Time[1]))
        ds = ds.mean("sample", keep_attrs=True)

        intvlstr, stackstr = self.interval_keys[Interval]
        if Interval=="day":
            xfilt = ds
        else:
            xfilt = ds.resample(time="1"+intvlstr).mean(keep_attrs=True) 
        xfilt.coords["year"] = xfilt.time.dt.year
        if Stack:
            sfunc = lambda x: x.groupby(Interval).mean(skipna=True, keep_attrs=True)
            xfilt.coords[Interval] = getattr(xfilt.time.dt, stackstr)
            xfilt = xfilt.groupby("year").apply(sfunc)

        self.output.clear_output()
        with self.output:
            try:
                pfunc(xfilt, Interval=Interval, Stack=Stack)
            except:
                display(self.error)
                
        #self.status.clear_output()

## plotting config

In [ ]:
plt.rc("font", **{"family": "normal", "weight": "bold", "size": 12})
plt.rc("axes", **{"xmargin": 0.01})

### plot means by observation type

In [ ]:
usfs_gpp = layer.layer.stats
smap_gpp = xrdataset["GPP_mean"]

rootzone = soil_moisture.filter_by_attrs(soil_zone="rootzone")
surface = soil_moisture.filter_by_attrs(soil_zone="surface")

In [ ]:
sel = "rootzone" # dummy selection

ds = rootzone if sel=="rootzone" else surface

is_sm = ds.filter_by_attrs(type="in situ")
a_sm = ds.filter_by_attrs(type="airborne")
s_sm = ds.filter_by_attrs(type="spaceborne")

In [ ]:
fig, ax0 = plt.subplots(figsize=(14,5))
fig.tight_layout()
ax1 = ax0.twinx()

for platform_type in ["in situ", "airborne", "spaceborne"]:
    psm = ds.filter_by_attrs(type=platform_type)
    pstack = xr.concat([psm[v] for v in psm], "mean").stack(cat=("mean", "sample"))
    pmean, pstd = pstack.mean("cat"), pstack.std("cat")   
    label = platform_type+" observations"
    pmean.plot(x="time", ax=ax0, label=label, add_legend=False)

smap_gpp.mean("sample").plot(x="time", ax=ax1)
(usfs_gpp.MEAN*0.001).plot(y="MEAN", use_index=True, ax=ax1)

ax0.grid('on', alpha=0.25)
ax0.set_title("Y1: "+sel+" average soil moisture | Y2: SMAP & USFS site productivity")
ax0.set_ylabel("m3/m3"); ax1.set_ylabel("g m-2 d-1")

In [ ]:
usfs_gpp.MEAN*0.001

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(18, 12))



for plotter in [(0, "rootzone", rootzone), (2, "surface", surface)]:
    axa, axb = axs[plotter[0]], axs[plotter[0]+1]
    
    arrays = []
    for name, dataset in plotter[2].items():
        data = dataset.mean("sample") if "sample" in dataset.dims else dataset
        if not all(data.isnull()):
            data.plot.line(ax=axa, x="time", label=name, add_legend=False)
            arrays.append(dataset)

    stack = xr.concat(arrays, "mean").stack(cat=("mean", "sample"))
    pmean, pstd = stack.mean("cat"), stack.std("cat")
    pmean.plot(x="time", ax=axb, label=plotter[1]+" mean")
    axb.fill_between(pmean.time.data, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    axa.set_ylabel(plotter[1]+" (m3/m3)"); axb.set_ylabel("(m3/m3)")
    axa.grid("on", alpha=0.25); axb.grid("on", alpha=0.25); 
    axa.legend(loc=2); axb.legend(loc=1)
    axa.set_xlabel(None); axb.set_xlabel(None)


axs[0].set_title("Rootzone soil moisture from in situ sources")
axs[2].set_title("Surface soil moisture from in situ sources")

## plot Daymet

In [ ]:
fig, ax0 = plt.subplots(figsize=(14,5))
fig.tight_layout()
ax1 = ax0.twinx()

daymet = xrds_mean.filter_by_attrs(source="Daymet")
prcp = daymet.prcp.resample(time="1m", keep_attrs=True).sum(keep_attrs=True).to_series() # sum precip over 1month
tmin = daymet.tmin.resample(time="1w", keep_attrs=True).mean(keep_attrs=True)            # avg tmin over 1week
tmax = daymet.tmax.resample(time="1w", keep_attrs=True).mean(keep_attrs=True)            # avg tmax over 1week

l1 = tmin.plot(ax=ax0, color="darkorange", label="Weekly avg min temperature")
l2 = tmax.plot(ax=ax0, color="darkgreen", label="Weekly avg max temperature")
ax1.bar(prcp.index, prcp, width=25, alpha=0.5, label="Monthly total precip")

ax0.set_ylim(-5, 45)
ax0.set_xlim("2005", "2010")
ax0.grid("on", alpha=0.5)
ax0.legend(loc="upper left")
ax0.set_ylabel(tmin.attrs["units"])
ax0.set_title("Daymet: Surface Meteorology 2005-2010")

ax1.legend()
ax1.set_ylabel(daymet.prcp.attrs["units"])

## plot soil moisture

#### from SMAP

In [ ]:
smap = xrds.filter_by_attrs(source="SMAP").sel(stat="Mean", drop=True)

fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(16, 8))

for p in [("SMAP_surface", 0), ("SMAP_rootzone", 0), ("GPP_mean", 1), ("NEE_mean", 1)]:
    ax = axs[p[1]]
    data = smap[p[0]]
    desc = data.attrs["description"]

    ptime = data.time.data
    pmean = data.mean("sample", keep_attrs=True)
    pstd = data.std("sample", keep_attrs=True)
    pmean.plot.line(x="time", ax=ax, label=desc, add_legend=False)
    ax.fill_between(ptime, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
axs[0].set_xlabel(None) 
axs[0].set_title("SMAP Soil Moisture")
axs[1].set_title("SMAP Gross Primary Production & Net Ecosystem Exchange")
axs[0].grid('on', alpha=0.25); axs[1].grid('on', alpha=0.25)
axs[0].legend(loc=0, framealpha=1); axs[1].legend(loc=0, framealpha=1)
axs[0].set_ylabel("m3/m3"); axs[1].set_ylabel("g m-2 d-1")

#### from AirMOSS

In [ ]:
airmoss = xrds.filter_by_attrs(source="AirMOSS").sel(stat="Mean", drop=True)

data = airmoss["AirMOSS_L4_rootzone"]
ptime = data.time.data
pmean = data.mean("sample", keep_attrs=True)
pstd = data.std("sample", keep_attrs=True)

fig = plt.figure(figsize=(16, 4))
ax = plt.gca()
pmean.plot.line(x="time", ax=ax, label=desc, add_legend=False)
ax.fill_between(ptime, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
ax.set_title("AirMOSS_L4_rootzone: "+data.attrs["description"])
ax.set_ylabel(data.attrs["units"])
ax.grid("on", alpha=0.25)

##### in case there is data from more than one airmoss dataset:

In [ ]:
airmoss = xrdataset.filter_by_attrs(source="AirMOSS").sel(stat="Mean", drop=True)

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16, 4))

for name, dataset in airmoss.items(): 
    ax = axs[0] if dataset.attrs["type"]=="airborne" else axs[1]
    desc = dataset.attrs["description"]
    pmean = dataset.mean("sample", keep_attrs=True)
    if not all(pmean.isnull()):
        pmean.plot.line(x="time", ax=ax, label=desc, add_legend=False)

    ax.set_title(None)
    ax.grid("on", alpha=0.25)

### all soil moisture datasets

In [ ]:
rootzone = xrdataset.filter_by_attrs(type="in situ", soil_zone="rootzone").sel(stat="Mean", drop=True)
surface = xrdataset.filter_by_attrs(type="in situ", soil_zone="surface").sel(stat="Mean", drop=True)
rootzone

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=1, sharex=True, figsize=(18, 12))

soil_moisture = xrdataset.filter_by_attrs(units="m3/m3")
rootzone = soil_moisture.filter_by_attrs(soil_zone="rootzone")
surface = soil_moisture.filter_by_attrs(soil_zone="surface")

for plotter in [(0, "rootzone", rootzone), (2, "surface", surface)]:
    axa, axb = axs[plotter[0]], axs[plotter[0]+1]
    
    arrays = []
    for name, dataset in plotter[2].items():
        data = dataset.mean("sample") if "sample" in dataset.dims else dataset
        if not all(data.isnull()):
            data.plot.line(ax=axa, x="time", label=name, add_legend=False)
            arrays.append(dataset)

    stack = xr.concat(arrays, "mean").stack(cat=("mean", "sample"))
    pmean, pstd = stack.mean("cat"), stack.std("cat")
    pmean.plot(x="time", ax=axb, label=plotter[1]+" mean")
    axb.fill_between(pmean.time.data, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    axa.set_ylabel(plotter[1]+" (m3/m3)"); axb.set_ylabel("(m3/m3)")
    axa.grid("on", alpha=0.25); axb.grid("on", alpha=0.25); 
    axa.legend(loc=2); axb.legend(loc=1)
    axa.set_xlabel(None); axb.set_xlabel(None)


axs[0].set_title("Rootzone soil moisture from in situ sources")
axs[2].set_title("Surface soil moisture from in situ sources")

In [ ]:
for i, p in enumerate(["AirMOSS_L2_3_surface", "AirMOSS_L2_3_rootzone", "AirMOSS_L4_rootzone"]):
    ax = axs[i]
    data = airmoss[p]
    desc = data.attrs["description"]

    ptime = data.time.data
    pmean = data.mean("sample", keep_attrs=True)
    pstd = data.std("sample", keep_attrs=True)
    pmean.plot.line(x="time", ax=ax, label=desc, add_legend=False)
    ax.fill_between(ptime, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    ax.set_title(None)
    ax.grid("on", alpha=0.25)

In [ ]:
airmoss = xrds.filter_by_attrs(source="AirMOSS").sel(stat="Mean", drop=True)

fig, axs = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(15, 8))
plt.subplots_adjust(hspace=0.000)

for i, p in enumerate(["AirMOSS_L2_3_surface", "AirMOSS_L2_3_rootzone", "AirMOSS_L4_rootzone"]):
    ax = axs[i]
    data = airmoss[p]
    desc = data.attrs["description"]

    ptime = data.time.data
    pmean = data.mean("sample", keep_attrs=True)
    pstd = data.std("sample", keep_attrs=True)
    pmean.plot.line(x="time", ax=ax, label=desc, add_legend=False)
    ax.fill_between(ptime, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    ax.set_title(None)
    ax.grid("on", alpha=0.25)

In [ ]:
fig, axs = plt.subplots(3, 1, sharex=True, figsize=(15,10))
ax1, ax2, ax3 = axs
ax4 = ax3.twinx()

In [ ]:
fig, ax0 = plt.subplots(figsize=(14,5))
fig.tight_layout()
ax1 = ax0.twinx()

daymet = xrds_mean.filter_by_attrs(source="Daymet")
prcp = daymet.prcp.resample(time="1m", keep_attrs=True).sum(keep_attrs=True)
l1 = daymet.tmin.plot(ax=ax0, color="darkgreen")
l2 = daymet.tmax.plot(ax=ax0, color="darkorange")
l3 = prcp.plot(ax=ax1, marker="o")

labels = ["Daily min temperature (deg C)", 
          "Daily max temperature (deg C)", 
          "Monthly total precip (mm/day)"]
ax1.legend(l1+l2+l3, labels, loc="upper right", framealpha=0.75)

ax0.grid("on", alpha=0.25)
ax0.set_ylabel(daymet.tmin.attrs["units"])
ax0.set_title("Daymet: surface meteorology")
ax1.set_ylabel(prcp.attrs["units"])
ax1.set_title(None)

# batch download big area

In [ ]:
urls = []

s = app.layers.samples.item()
for i, r in s.iterrows():
    urls.append(r.samp.dl)

print(len(urls))

In [ ]:
def get_sample_xr(df, id, lat, lon):
    """ """

    d = ["sample"]                          
    s = xr.DataArray(data=[id], dims=d) # get sample, lat, lon xr arrays
    y = xr.DataArray(data=[lat], coords=[s], dims=d, attrs=latatts)
    x = xr.DataArray(data=[lon], coords=[s], dims=d, attrs=lonatts)

    ds = {}
    for dataset in df.columns:
        a = smvdatasets.loc[dataset].to_dict()
        if a["source"] not in disabled_sources:
            split_column = split_pd(df[dataset])
            ds[dataset] = pd_to_xr(dataset, split_column)

    xds = xr.merge(ds.values())                          # merge to one xr
    xds = xds.assign_coords(lat=y, lon=x)                # add coord arrays
    
    return(xds)


outdir = "/home/jack/tmp/"
for i, u in enumerate(urls):
    print("Processing "+str(i))
    
    lat, lon = [float(c.split("&")[0]) for c in u.split("=")[1:3]]
    out = outdir+str(i)+"_"+str(lat)+"_"+str(lon)+"."
    
    response = requests.get(u, cookies=auth)
    rtxt = response.text
    with open(out +"txt", "w") as f:
        f.write(rtxt)
    df = txt_to_pd(rtxt)                       # read to df
    xrds = get_sample_xr(df, i, lat, lon)                # get xr dataset
    
    comp = dict(zlib=True, complevel=5)
    encoding = {var: comp for var in xrds.data_vars}
    xrds.to_netcdf(out +"nc", encoding=encoding)

In [ ]:
def plotSoilMoisture(xrds, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(14, 6)) 
    fig.tight_layout()
    ax1 = ax0.twinx()

    for plotter in :
        
        label = plotter[0]+" mean"
        
        if Stack:
            yrs = np.unique(xrds.year)
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").sel(year=np.unique(stack.year))
            pmean.mean("year").plot.line(x=Interval, ax=ax0, label=label)
        else:
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").plot(x="time", ax=ax0, label=label)

        #ax.fill_between(pmean.time.data, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)

        ax0.set_ylabel()
        ax0.grid("on", alpha=0.25)
        ax0.legend(loc=2)
        ax0.set_title(None)
        ax0.set_xlabel(Interval)
        
    plt.show()

def plotDaymet(xrds, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(10, 4.5))
    fig.tight_layout()
    ax1 = ax0.twinx()
   
    prcp = xrds.prcp
    tmin = xrds.tmin
    tmax = xrds.tmax
    ylabels = ("degrees Celsius", "mm/day")
    
    if (Stack) & (Interval!="year"):
        l1 = prcp.mean("year").plot.line(x=Interval, ax=ax0, color="darkorange", label="avg rate of precip")
        l2 = tmin.mean("year").plot.line(x=Interval, ax=ax0, color="darkgreen", label="avg min temp")
        l3 = tmax.mean("year").plot.line(x=Interval, ax=ax1, color="purple", label="avg max temp", linestyle="--")
    else:
        l1 = prcp.plot(ax=ax0, color="darkorange", label="avg rate of precip")
        l2 = tmin.plot(ax=ax0, color="darkgreen", label="avg min temp")
        l3 = tmax.plot(ax=ax1, color="purple", label="avg max temp", linestyle="--")

    ax0.grid("on", alpha=0.5)
    ax0.legend(loc="upper left"); ax1.legend()
    ax0.set_ylabel(ylabels[0]); ax1.set_ylabel(ylabels[1])

    plt.show()


def plotSoilMoisture(xrds, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(10, 4.5)) 
    fig.tight_layout()
    ax1 = ax0.twinx()

    for plotter in [
        ("surface", xrds.filter_by_attrs(soil_zone="surface")), 
        ("rootzone", xrds.filter_by_attrs(soil_zone="rootzone"))]:
        
        label = plotter[0]+" mean"
        
        if (Stack) & (Interval!="year"):
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").sel(year=np.unique(stack.year))
            pmean.mean("year").plot.line(x=Interval, ax=ax0, label=label)
        elif (Stack) & (Interval=="year"):
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").plot(x="year", ax=ax0, label=label) 
        else:
            stack = xr.concat(plotter[1].values(), "mean")
            pmean = stack.mean("mean").plot(x="time", ax=ax0, label=label)

        #ax.fill_between(pmean.time.data, (pmean-pstd), (pmean+pstd), color="gray", alpha=0.2)
    
    ax0.set_ylabel("soil moisture volume (m3/m3)")
    ax0.grid("on", alpha=0.25)
    ax0.legend(loc=2)
    ax0.set_title(None)
    ax0.set_xlabel(Interval)
        
    plt.show()

    
def plotDataset(plotters, ylabels, Interval=None, Stack=False):
    """ """
    
    fig, ax0 = plt.subplots(figsize=(14, 6))
    fig.tight_layout()
    ax1 = ax0.twinx()
    
    axs = [ax0,ax1]
    if Stack:
        pfunc = lambda p: p[0].mean("year").plot.line(x=Interval, ax=axs[p[1]], color=p[2], label=p[3])
    else:
        pfunc = lambda p: p[0].plot(ax=axs[p[1]], color=p[2], label=p[3])

    for p in plotters:
        pfunc(p)
    
    ax0.grid("on", alpha=0.5)
    ax0.legend(loc="upper left"); ax1.legend()
    #ax0.set_xlim(xrds.time.data[0], xrds.time.data[-1])
    ax0.set_ylabel(ylabels[0]); ax1.set_ylabel(ylabels[1])

    plt.show()

dataset_keys = {
    
    "Daymet": (
        lambda x: x.filter_by_attrs(source="Daymet"), 
        [lambda x: x.prcp,
         lambda x: x.tmin,
         lambda x: x.tmax],
        [[0, "darkorange", "avg rate of precip"],
         [0, "darkgreen", "avg min temp"],
         [1, "purple", "avg max temp"]],
        ("degrees Celsius", "mm/day")), 
    
    "Soil moisture": (
        lambda x: x.filter_by_attrs(units="m3/m3"), 
        [lambda x: xr.concat(x.filter_by_attrs(soil_zone="surface").values(), "mean").mean("mean"), 
         lambda x: xr.concat(x.filter_by_attrs(soil_zone="rootzone").values(), "mean").mean("mean")],
        [[0, "darkorange", "surface"], 
         [0, "darkgreen", "rootzone"]],
         ("soil moisture volume (m3/m3)", None))}

interval_keys = {
    "day": ("d", "dayofyear"), 
    "week": ("w", "week"), 
    "month": ("m", "month"), 
    "year": ("y", "year")}

Dataset = "Daymet"
Interval = "day"
Stack = False
#Time = 

dfilter, grabbers, plotters, ylabels = dataset_keys[Dataset]
ds = dfilter(p.xr)
#ds = ds.sel(time=slice(Time[0], Time[1]))
ds = ds.mean("sample", keep_attrs=True)

intvlstr, stackstr = interval_keys[Interval]
xfilt = ds.resample(time="1"+intvlstr).mean(keep_attrs=True)
xfilt.coords["year"] = xfilt.time.dt.year

sfunc = lambda x: x.groupby(Interval).mean(skipna=True, keep_attrs=True)
xfilt.coords[Interval] = getattr(xfilt.time.dt, stackstr)
xfilt = xfilt.groupby('year').apply(sfunc)

# -----------------------------------------------------------------------

plotters2 = [tuple([grbr(xfilt)]+pltr) for grbr, pltr in zip(grabbers, plotters)]
plotDataset(plotters2, ylabels, Interval=Interval, Stack=Stack)